In [1]:
def gen_proxy(proxy_id: int) -> str:
    super_proxy_url = ('http://%s-session-%s:%s@zproxy.lum-superproxy.io:%d' %
                       ('lum-customer-hl_1aa52067-zone-checknft_unlimited',
                        proxy_id,
                        'to08bx2tnx2u',
                        22225))
    return super_proxy_url


ALL_PROXY = [gen_proxy(i) for i in range(100)]
ALL_PROXY.append('self_url')

In [45]:
import sys
import eventlet
from eventlet.green.urllib import request
import random
import socket

super_proxy = socket.gethostbyname('zproxy.lum-superproxy.io')

class SingleSessionRetriever:

    url = "http://%s-session-%s:%s@"+super_proxy+":%d"
    port = 22225

    def __init__(self, username, password, requests_limit, failures_limit):
        self._username = username
        self._password = password
        self._requests_limit = requests_limit
        self._failures_limit = failures_limit
        self._reset_session()

    def _reset_session(self):
        session_id = random.random()
        proxy = SingleSessionRetriever.url % (self._username, session_id, self._password,
                                              SingleSessionRetriever.port)
        proxy_handler = request.ProxyHandler({'http': proxy, 'https': proxy})
        self._opener = request.build_opener(proxy_handler)
        self._opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36')]
        self._requests = 0
        self._failures = 0

    def retrieve(self, url, timeout):
        while True:
            if self._requests == self._requests_limit:
                self._reset_session()
            self._requests += 1
            try:
                timer = eventlet.Timeout(timeout)
                result = self._opener.open(url).read()
                timer.cancel()
                return result
            except:
                timer.cancel()
                self._failures += 1
                if self._failures == self._failures_limit:
                    self._reset_session()


class MultiSessionRetriever:

    def __init__(self, username, password, session_requests_limit, session_failures_limit):
        self._username = username
        self._password = password
        self._sessions_stack = []
        self._session_requests_limit = session_requests_limit
        self._session_failures_limit = session_failures_limit

    def retrieve(self, url, timeout, parallel_sessions_limit):
        pool = eventlet.GreenPool(parallel_sessions_limit)
        return self._retrieve_single(url, timeout)

    def _retrieve_single(self, url, timeout):
        if self._sessions_stack:
            session = self._sessions_stack.pop()
        else:
            session = SingleSessionRetriever(self._username, self._password,
                                             self._session_requests_limit, self._session_failures_limit)
        body = session.retrieve(url, timeout)
        self._sessions_stack.append(session)
        return body

def output(url, body):
    print('anona')

n_total_req = 1
req_timeout = 10
n_parallel_exit_nodes = 1
switch_ip_every_n_req = 100
max_failures = 3


In [46]:
retriver = MultiSessionRetriever('lum-customer-hl_1aa52067-zone-checknft_unlimited', 'to08bx2tnx2u', switch_ip_every_n_req, max_failures)

In [47]:
data = retriver.retrieve(
    ["https://lh3.googleusercontent.com/Ei16b_P-1MW44Z5mUTmq_s7xJrQSv6Mz1EdNE1HWQDHg9Dx3XuXZ1umXOtsMwn1flYQRX-zhLKj7Da4YiHNcOrDMhpGYxH-rjvW9-w"] * n_total_req, req_timeout, n_parallel_exit_nodes)

In [48]:
data

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x02\x00\x08\x02\x00\x00\x00{\x1aC\xad\x00\x00\x00\x03sBIT\x08\x08\x08\xdb\xe1O\xe0\x00\x00\x00\x06bKGD\x00\xff\x00\xff\x00\xff\xa0\xbd\xa7\x93\x00\x00 \x00IDATx\x9c\xec\xbdy|]Wu\xf7\xbd\xd6\x1e\xce9w\xbe\x9a%K\x9e\xed\xc4v\x1c\x823@f\x92&\x04JH\x08Px)e(t\x80\xf6\xe1y\x1eZ\xde2\xf4\xe9\x00\x1d\xa0\xb4L\xa5}\xa0@K(Ch\x98\xda\x86\xa6\x01\x9a0\x96@\xc8\xe8$\x8e\xe7Y\xd6,]\xdd\xf9\x0c{\xef\xf5\xfcq\xee\x95ey\x94-\xc9\x89\xb4\xbf\xd6G\xba\xf7\xea\xdc}\xce\xb9\xb6\xd7o\xef\xb5\xd6^\x0b\xe9\x0bw\x02\x81\xc5b\xb1X\x96\x16\x08\xccZ\x7f\x8b\xc5bY\x8a\x10\xb0\xf3}\t\x16\x8b\xc5b9?X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x00\x8b\xc5bY\xa2X\x01\xb0X,\x96%\x8a\x15\x0